In [ ]:
# functions from https://www.thepythoncode.com/article/extracting-and-submitting-web-page-forms-in-python
def get_all_forms(url):
    """Returns all form tags found on a web page's `url` """
    # GET request
    res = session.get(url)
    # for javascript driven website
    # res.html.render()
    soup = bs(res.html.html, "html.parser")
    return soup.find_all("form")

def get_form_details(form):
    """Returns the HTML details of a form,
    including action, method and list of form controls (inputs, etc)"""
    details = {}
    # get the form action (requested URL)
    action = form.attrs.get("action").lower()
    # get the form method (POST, GET, DELETE, etc)
    # if not specified, GET is the default in HTML
    method = form.attrs.get("method", "get").lower()
    # get all form inputs
    inputs = []
    for input_tag in form.find_all("input"):
        # get type of input form control
        input_type = input_tag.attrs.get("type", "text")
        # get name attribute
        input_name = input_tag.attrs.get("name")
        # get the default value of that input tag
        input_value =input_tag.attrs.get("value", "")
        # add everything to that list
        inputs.append({"type": input_type, "name": input_name, "value": input_value})
    # put everything to the resulting dictionary
    details["action"] = action
    details["method"] = method
    details["inputs"] = inputs
    return details

In [ ]:
def pretty_print(form_details,show_hidden=True):
    for k in form_details.keys():
        print(k)
        res = form_details[k]
        if type(res) == list:
            if show_hidden == False and list(set(list(map(type,res))))[0]==dict:
                res = list(filter(lambda d: d['type']!='hidden',res))
            for v in res:
                print('\t',v)
        else: print('\t',res)
        print()

In [ ]:
# url = 'https://notifications.hse.gov.uk/COMAH2015/Search.aspx'
# forms = get_all_forms(url)
# for n,f in enumerate(forms, start=1):
#     form_details = get_form_details(f)
#     print("="*50, f"form #{i}", "="*50)
#     pretty_print(form_details,show_hidden=False)

In [18]:

# # page = s.get(url)#,headers=headers)
# # soup = bs(page.content)
# #     data['__VIEWSTATE'] = soup.select_one('#__VIEWSTATE')['value']
# #     data['__VIEWSTATEGENERATOR'] = soup.select_one('#__VIEWSTATEGENERATOR')['value']
# #     data['__EVENTVALIDATION'] = soup.select_one('#__EVENTVALIDATION')['value']
#     open_page = s.post(url, data=data)#,headers=headers)
# #     open_page = s.get(url,headers=headers)

# with open("requests_results.html", "w",encoding="utf-8") as f:
#     f.write(open_page.text)

# Import modules

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

# Set variables

In [16]:
url = 'https://notifications.hse.gov.uk/COMAH2015/Search.aspx'
data = {'ctl00$ContentPlaceHolder1$Postcode':'M30 9LL',
        'ctl00$ContentPlaceHolder1$btnSearch': 'Search'}

In [17]:
# headers = {'authority': 'notifications.hse.gov.uk',
# 'method': 'POST',
# 'path': '/comah2015/Search.aspx',
# 'scheme': 'https',
# 'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
# 'accept-encoding': 'gzip, deflate, br',
# 'accept-language': 'en-US,en;q=0.9',
# 'cache-control': 'max-age=0',
# 'content-length': '935',
# 'content-type': 'application/x-www-form-urlencoded',
# 'cookie': 'optInGoogleTracking=false; ASP.NET_SessionId=ml2h0awsujzsnxfe3e5bt1ua',
# 'origin': 'https://notifications.hse.gov.uk',
# 'referer': 'https://notifications.hse.gov.uk/comah2015/Search.aspx',
# 'sec-ch-ua': '"Chromium";v="94", "Google Chrome";v="94", ";Not A Brand";v="99"',
# 'sec-ch-ua-mobile': '?0',
# 'sec-ch-ua-platform': '"Windows"',
# 'sec-fetch-dest': 'document',
# 'sec-fetch-mode': 'navigate',
# 'sec-fetch-site': 'same-origin',
# 'sec-fetch-user': '?1',
# 'upgrade-insecure-requests': '1',
# 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36'}

# Define functions

## Submit information to Form in URL

In [18]:
def submit form(url,data):
    response = requests.get(url)
    
    soup = bs(response.content)
    data['__VIEWSTATE'] = soup.select_one('#__VIEWSTATE')['value']
    data['__VIEWSTATEGENERATOR'] = soup.select_one('#__VIEWSTATEGENERATOR')['value']
    data['__EVENTVALIDATION'] = soup.select_one('#__EVENTVALIDATION')['value']

    response = requests.post(url,data=data,headers=headers)
    doc = bs(response.text, 'html.parser')
    return doc

In [ ]:
submit form(url,data)

In [32]:
# with requests.Session() as s:
#     response = s.get(url)

#     soup = bs(response.content)
#     data['__VIEWSTATE'] = soup.select_one('#__VIEWSTATE')['value']
#     data['__VIEWSTATEGENERATOR'] = soup.select_one('#__VIEWSTATEGENERATOR')['value']
#     data['__EVENTVALIDATION'] = soup.select_one('#__EVENTVALIDATION')['value']
    
#     response = s.post(url,data=data,headers=headers)
#     doc = bs(response.text, 'html.parser')

# Peform check

In [85]:
def check_results_page(doc):
    if 'You are in: COMAH 2015 » Search' in doc.text:
        res = 'ERROR - page is "Search"'
    elif 'You are in: COMAH 2015 » Reults' in doc.text:   
        res = 'OK - page is "Results"'
    return res

In [86]:
check_results_page(doc)

'ERROR - page is "Search"'

# Get links from results page

In [6]:
def get_links_from_results(result_html):
    res=[]
    for link_element in doc.find_all('a'):
        link = link_element.get('href')
        if link != None and 'PublicInformation' in link:
            link = 'https://notifications.hse.gov.uk/comah2015/'+link
            res.append(link)
    return res

In [ ]:
links = get_links_from_results(doc)

# Extract tables from links

continue with a single 'response' for now...

In [42]:
response = requests.get(links[0])

In [8]:
doc = bs(response.text,'html.parser')

In [7]:
# find tab id and corresponding name for given link
table_names = {}
for tab_header in doc.find("div", {"class": "ajax__tab_header"}):
    try:    
        tab_id = tab_header.get('id').rsplit('_',1)[0]
        table_name = tab_header.get_text()
        table_names[tab_id] = table_name
        a = tab_id
    except: pass

TypeError: 'NoneType' object is not iterable

In [8]:
table_names

{}

In [254]:
for div in doc.find_all('div',{'class':'ajax__tab_panel'}):
    print(div.get('id'))
    table_html = div.find('table')

ContentPlaceHolder1_tcInfo_tpPanel1
ContentPlaceHolder1_tcInfo_tpPanel2
ContentPlaceHolder1_tcInfo_tpPanel3
ContentPlaceHolder1_tcInfo_tpPanel4
ContentPlaceHolder1_tcInfo_tpPanel5


In [81]:
for table in doc.find_all('table'):
    print('-'*100)
    print(table.find_all('td'))
#     print(table_names)
#     print('-'*100)
#     for row in table.find_all('tr'):
#         print(row)

# Main program

In [ ]:
# for n,form in enumerate(forms,start=1):
#     print('Form',n,'input elements')
#     print('type',' '*6,'attributes',' '*40,'name','\n'+'-'*100)
#     for i in form.find_all("input"):
#         input_type = i['type']
#         input_attrs = list(i.attrs.keys())
#         s1 = input_type
#         s2 = input_attrs
#         s3 = i['name']
#         pad12 = ' '*(10-len(s1))
#         pad23 = ' '*(50-(len(str(s2))))
#         print(s1,pad12,s2,pad23,s3)